In [ ]:
#!/usr/bin/env python3
# -*- coding: UTF-8 -*-

import os
import sys
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
from opencc import OpenCC

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
#os.environ['AUTOGRAPH_VERBOSITY']='10'
TKIPDATA="./"
cc = OpenCC('t2s')

def read_file(f):
    lines = f.read().splitlines()
    return lines

def pos_analysis(lst, ws, pos, ner):
    word_sentence_list = ws(lst, sentence_segmentation=True)
    pos_sentence_list = pos(word_sentence_list)
    
    return(word_sentence_list, pos_sentence_list)

def print_pos(sentence_list, word_sentence_list,  pos_sentence_list, ofp, simpfp=None):
    for i, sentence in enumerate(sentence_list):
        #print(sentence, word_sentence_list[i],  pos_sentence_list[i])
        print_word_pos_sentence(sentence_list, word_sentence_list[i],  pos_sentence_list[i], ofp, simpfp)

# Show results
def print_word_pos_sentence(sentence_list, word_sentence_list, pos_sentence_list, ofp, simpfp):
    assert len(word_sentence_list) == len(pos_sentence_list)
    for word, pos in zip(word_sentence_list, pos_sentence_list):
        if "CATEGORY" not in pos:
            prword = word.strip('\n')
            sprword = cc.convert(prword)
            print(f"{prword}", end=" ")
            ofp.write(f"{prword} ")
            #simpfp.write(f"{sprword} ")
            
def tkip_data():
    if not os.path.isdir(TKIPDATA):
        print("Downloading CKIP data...")
        # Download data
        os.makedirs(TKIPDATA)
        data_utils.download_data(TKIPDATA)

    print("Loading CKIP data...")
    # Load model
    ws = WS(TKIPDATA+"data")
    pos = POS(TKIPDATA+"data")
    ner = NER(TKIPDATA+"data")
    
    return (ws, pos, ner)

(ws, pos, ner) = tkip_data()

while(True):
    txtfile=input("Input text file: ")
    posfile=os.path.splitext(txtfile)[0]+'.pos'
    #simpfile=os.path.splitext(txtfile)[0]+'.simp'
    #print("Input: ", txtfile, "Output: ", posfile, "簡體中文: ", simpfile)
    print("Input: ", txtfile, "Output: ", posfile)
    print()
    
    fp = open(txtfile, encoding="utf8")
    ofp= open(posfile, "w", encoding="utf8")
    #simpfp = open(simpfile, "w", encoding="utf8")
    lines = read_file(fp)
    for l in lines:
        print(l)

    print()
    print("-----------------------------")
    (wl, pl) = pos_analysis(lines, ws, pos, ner)
    #print_pos(lines, wl, pl, ofp, simpfp)
    print_pos(lines, wl, pl, ofp)
    print()
    print("-----------DONE--------------")
    fp.close()
    ofp.close()
    #simpfp.close()
    
# Release model
del ws
del pos
del ner



Loading CKIP data...
Input text file: D:\USER\Downloads\calign\Forced_Aligner_Mandarin\test8\04232019_2.txt
Input:  D:\USER\Downloads\calign\Forced_Aligner_Mandarin\test8\04232019_2.txt Output:  D:\USER\Downloads\calign\Forced_Aligner_Mandarin\test8\04232019_2.pos

@這種錄音器錄音啊幹嘛。
我們的錄音筆比較小一點
也是這個這個牌子對不對
恩
不是嗎
我我我沒有看牌子欸
喔你沒有沒有看牌子喔好
那那個可以開始囉
好我們上禮拜講到
@@還沒有同意
喔再來這邊好我先跳到上禮拜的
恩哼
有這邊
就是上禮拜我們把它的phonetics的數據啊它的那些的都講完了
然後我先就是跟大家複習一下
反正就大概是這樣子它就是預測說在underlying structure越多independent mora的vowel或者是consonant他們單獨的duration就越長
所以在這這三個語言我們可以看到就是有一個mora它的duration都是一
然後有一個mora加一個shared mora的duration大概就是一點五。
那兩個mora的話它的duration就是二
這樣大概
然後然後在因為有一些語言它的coda是有weight
然後在有一些語言它的coda是沒有weight
那在coda是有weight的語言裡面我們的underlying structure它的coda就是會連到前面前面母音的mora
所以那個consonant就會有一個shared mora
然後被share的mora的那個母音它的mora也是也是那個shared mora
這樣
所以她現在就提出呃一堆constraint然後來去用優學理論解釋這些呃跨語言的
就是各種語言的underlying morae structure
那呃首先因為我們看的三個語言就是呃
阿拉伯語，然後馬拉雅拉姆語，還有呃印度語，他們三這三個語言的coda都是有mora的，所以他就是提出一個constraint
叫做morae coda。
它的定義，就是說全部的coda co

@ 這 種 錄音器 錄音 啊 幹嘛 我們 的 錄音筆 比較 小 一點 也 是 這 個 這 個 牌子 對 不 對 恩 不 是 嗎 我 我 我 沒有 看 牌子 欸 喔 你 沒有 沒有 看 牌子 喔 好 那 那 個 可以 開始 囉 好 我們 上 禮拜 講到 @@ 還 沒有 同意 喔 再 來 這邊 好 我 先 跳到 上 禮拜 的 恩哼 有 這邊 就 是 上 禮拜 我們 把 它 的 phonetics 的 數據 啊 它 的 那些 的 都 講完 了 然後 我 先 就 是 跟 大家 複習 一下 反正 就 大概 是 這樣子 它 就 是 預測 說 在 underlying structure 越 多 independent mora 的 vowel 或者 是 consonant 他們 單獨 的 duration 就 越 長 所以 在 這 這 三 個 語言 我們 可以 看到 就 是 有 一 個 mora 它 的 duration 都 是 一 然後 有 一 個 mora 加 一 個 shared mora 的 duration 大概 就 是 一點五 那 兩 個 mora 的話 它 的 duration 就 是 二 這樣 大概 然後 然後 在 因為 有 一些 語言 它 的 coda 是 有 weight 然後 在 有 一些 語言 它 的 coda 是 沒有 weight 那 在 coda 是 有 weight 的 語言 裡面 我們 的 underlying structure 它 的 coda 就 是 會 連到 前面 前面 母音 的 mora 所以 那 個 consonant 就 會 有 一 個 shared mora 然後 被 share 的 mora 的 那 個 母音 它 的 mora 也 是 也 是 那 個 shared mora 這樣 所以 她 現在 就 提出 呃 一 堆 constraint 然後 來去 用 優學 理論 解釋 這些 呃 跨 語言 的 就 是 各 種 語言 的 underlying morae structure 那 呃 首先 因為 我們 看 的 三 個 語言 就 是 呃 阿拉伯語 然後 馬拉雅拉姆語 還 有 呃 印度語 他們 三 這 三 個 語言 的 coda 都 是 有 mora 的 所以 他 就 是 提出 一 個 constraint 叫做 morae cod

 的 定義 就 是 coda 不行 呃 coda 不行 有 自己 一 個 mora 對 喔 然後 在 german 就是 之前 我們 就 是 phonology 的 時候 也 有 學 過 ＠＠＠ 就 是 就 是 ＠＠＠ 他 特別 的 地方 就 是 在於 它 是 他 可以 當作 前面 那 個 syllable 的 coda 也 可以 當作 後面 syllable 的 onset 所以 說 他 的 才 會 在 這邊 然後 連 到 前面 的 syllable 就 是 他 跨 syllable 所以 就 是 我們 就 可以 看到 馬拉雅拉姆語 的 coda 就 是 都 是 沒有 自己 一 個 mora 都 是 weightless 不管 在 ＠＠＠ 還是 non @@@ 都 是 一樣 好 那 這 個 的話 就 是 single c 就 是 這 個 就 是 非 ＠＠＠ 的 情況 非 ＠＠＠ 的 情況 的話 我們 就 是 也 是 可以 看到 他 的 coda 也 不會 有 自己 一 個 mora 然後 他 也 不行 沒有 mora 就 是 用 同樣 一 個 constraint ranking 都 可以 解決 好 所以 他 就 是 用 這些 這 四 個 con 這 幾 個 constraint 他 用 這 四 個 constraint 然後 去 做 不同 的 排列 就 可以 得到 不同 的 語言 就 是 這 就 是 他 的 typological prediction 所以 他 就 認為 呃 用 這 四 個 constraint 可以 得到 這些 possible 的 Morae structure 然後 type a 就 是 印度語 印度語 的話 就 是 coda 都 是 有 weight 他 coda 都 是 有 weight 所以 coda 有 自己 一 個 mora 那 type b 的話 就 是 coda 都 是 沒有 weight 然後 coda 就是 一定 要 跟 前面 母音 去 share mora Type c 的話 就 是 CVC 的 時候 C 是 有 weight 可是 CVVC 的 時候 沒有 weight 然後 type d 的話 就 是 CVC 的 時候 C 有 weight 然後 可是 他 CVVC 是 呃 Ｖ 會 少 一 個 mora 這樣 那 

 那 個 borrowed items need to be adapted into a new system 這樣子 好 那 我們 之後 在 講 就 借借 的 語言 的 時候 就 是 loan words 假如 說 我們 從 英文 借過來 那 個 語言 那 英文 我們 把 它 叫做 L2 那 就 是 中文 的 的話 我們 就 叫做 L1 這樣子 好 那 課本 那 個 文章 就 有 就 是 講到 這些 例子 像 呃 從 如果 是 從 法文 呃 英文 從 法文 借 詞 過來 的話 那 這 兩 個 字 那 個 along 跟 @@ 然後 跑到 英文 去 的話 就 會 變成 alen  跟 nuance 這樣子 那 他 這邊 我們 就 可以 觀察到 一 個 規則 說 因為 英文 中 他 的 那 個 phonemic nasal vowels 是 沒有 的 然後 也 沒有 那 個 rounded vowels @ 所以 那 個 @ 就 會 跑到 英文 去 就 會 變成 @ 然後 那 個 @ 在 法文 裡面 然後 借過去 英文 的話 就 會 變成 @ 這樣子 好 那 另外 一 個 例子 都 是 從 英文 借 就 是 借 字 到 日文 去 那 因為 日文 他 沒有 那 個 onsets 那 個 @ 這樣 @ 然後 也 沒有 那 個 coda @ 所以 他 都 會 把 中間 夾 一 個 母音 進去 所以 像 那 個 desk 就 會 變成 desuku 然後 credit 就 會 變成 kurejitto 這樣子 對 好 那 我們 從 觀察 loan words 的 中間 我們 可以 看到 什麼 樣 就 是 研究出 什麼 樣 的 現象 呢 然後 呃 文章 就 有 提到 說 觀察 loan word 我們 可以 就 是 weigh phonological constraint 就 是 找到 evidence 因為 如果 phonological 他 沒有 constraint 的話 那 loan word 他 就 不用 adapt 對 不 對 就是 假如 說 我們 從 phonological 在 一 個 語言 裡面 他 沒有 任何 的 限制 的話 那 我們 語言 借過來 借過去 都 都 不用 適應 他 那 個 就 是 L1 的 的 規則 了 所以 我們 觀察 loan wor